# SSD: Single Shot MultiBox Detector

![chart](./ssd_chart.png)

## Abstract

- 새로운 방식 SSD 사용
- SSD는 proposal generation과 그 후처리를 필요로 하지 않는 하나의 네트워크
- 우리 것이 짱이다 주장

# 1. Introduction

Faster RCNN == FRCNN

- 현재 sota FRCNN 은 너무 느림(7FPS)


- 빠르게 하기 위한 FRCNN 변형은 모두 정확도를 현저히 떨굼(YOLO - (45 FPS with mAP 63.4%))


- bbox proposal을 뽑아내는 부분과 그 후처리를 없애버림 (59 FPS with mAP 74.3%)


- 여러가지 크기의 feature map에 Conv layer 처리를 하고 합치는 방식을 사용함

# 2. SSD

## 2.1 Model

##### Structure

![fig2](./ssd_fig2.png)

YOLO 는 최종 특징맵에만 경계 박스와 클래스 정보가 있는데 비해 SSD 는 여러 레이어에 정보가 분산

class 수는 pascal voc 기준 background 포함 21개

+4 는 (x, y, w, h) bbox 데이터 의미

k x (Classes + 4) 형태인데 k 는 default box 개수(aspect ratio)를 의미

.

38x38x(4x(Classes+4)) = 5776x(Classes+4) - conv4_3

19x19x(6x(Classes+4)) = 2166x(Classes+4) - conv7

10x10x(6x(Classes+4)) = 600x(Classes+4) - conv8_2

 5x 5x(6x(Classes+4)) = 150x(Classes+4) - conv9_2

 3x 3x(4x(Classes+4)) = 36x(Classes+4) - conv10_2

 1x 1x(4x(Classes+4)) = 4x(Classes+4) - conv11_2


![fig1](./ssd_fig1.png)

feature map을 여러 개의 크기로 만들어서, 큰 map에서는 작은 물체의 검출을, 작은 map에서는 큰 물체의 검출을 함.

이러한 방식은 위치 추정 및 입력 이미지의 resampling을 없애면서도 정확도 높은 결과를 도출함

여러가지 미리 정의된 박스를 default box라 부름

총 6개까지 사용하는데 1:1, 1:1, 1:2, 2:1, 1:3, 3:1 를 사용함

scale 관련식은 추후 2.2 절에서 설명!

##### Example

예를 들어 default box가 3일 때는 다음 그림들 처럼 동작

![st1](./ssd_structure.png)

![st3](./ssd_structure3.png)

5x5x(3x(21+4)) 형태인데 21개에서는 softmax를 거쳐 21개의 probability를 뽑아냄.

4에서는 bbox 변화량을 출력으로 뽑아냄. x, y, w, h 모두 bbox regr.

##### NMS(non-maximum-suppression)

![st3](./ssd_nms.png)

##### bbox_regr

![](ssd_bbox_regr.png)

## 2.2 Training

### Matching Strategy

yolo, rpn, multibox 등등과 같이 ssd 도 학습을 위해 정해진 output 형태가 있어야 합니다.

- IOU가 0.5 이상일 경우 학습을 진행
- 0.5 이상인 녀석이 없으면 최대 IOU를 가진 애로 학습

Before | After
:---:|:---:
![](./ssd_default_box1.png) | ![](./ssd_default_box2.png)

### Training Objective(LOSS)

#### All Loss

![](./ssd_loss1.png)

전체 loss는 위와 같이 정의된다.

bbox에 대한 loc loss와 classifier loss의 합 형태이다.

.

a(알파) : loc에 대해 설정할 비율 cross validation으로 측정해본 결과 1이 적합

N : 일치하는 default box 의 수. 0일 경우 전체 loss도 0으로 설정

x : class 별 예측 confidence(0~1)

c : class 별 confidence

l : 예측한 bbox의 dx, dy, dw, dh

g : ground truth bbox 의 dx, dy, dw, dh

##### cross validation

![](./ssd_cross_val.png)

#### localization loss

![](./ssd_loss2.png)

loc loss에는 smooth L1 loss를 적용했음.

faster rcnn에서의 bbox regression 방식과 동일함

##### smooth_l1

![](ssd_smooth_l1.png)

#### classification(confidence) loss

![](./ssd_loss3.png)

softmax + categorical cross entropy 형태입니다.

### Choosing Scales and Aspect Ratios for Default Boxes

![](ssd_choose_scale.png)

default box의 크기는 위 식을 이용하여 구함

k : default box의 number

Smin : 0.2
Smax : 0.9

즉 k 값에 따라 s(scale) 값은 0.2 ~ 0.9까지 올라감

값은 약 [0.2, 0.32, 0.44, 0.56, 0.68, 0.80, 0.9]

ar은 종횡비로 [1, 2, 3, 1/2, 1/3]

![](ssd_choose_scale2.png) 
![](ssd_choose_scale3.png) 

위 두 식으로 width와 height를 구하고

![](ssd_choose_scale4.png)

이 식으로 default bbox를 하나 더 추가합니다.

### Hard Negative Mining

마지막으로 confidence loss가 높은 순으로 디폴트 박스를 정렬하여 positive와 negative의 비율이 1:3이 되도록 confidence loss가 높은것만 선별하여 학습 

오브젝트에 대한 신뢰 로스(오류)가 큰것만을 로스를 줄이는 방향으로 학습시키다 보면 결국 모든 샘플들의 신뢰 로스가 점차 낮아짐

이 방법을 사용하면 더 빠르고 안정적인 학습이 이뤄지는 것을 확인

### Data Augmentation

다음 중 하나를 랜덤으로 뽑아서 학습을 시켜서 data augmentation을 수행하였습니다.
- 입력 이미지를 그대로 넣기
- 최소 IoU가 0.1, 0.3, 0.5, 0.7, 0.9 인 샘플 패치를 넣기
- random patch (random cropping)

# 3. Result

### 3.1 Pascal VOC 2007 test set

![](ssd_result1.png)
![](ssd_result2.png)

# YOLO V2?

FC 를 제거하여 fully-convolutional model 로 변화

hard negative(절대로 오브젝트가 있을 수 없는 위치) 를 학습하지 않도록 조절

width, height 를 직접 예측했던 전 버전과 다르게 prior(SSD의 디폴트 박스, R-CNN의 앵커 박스)를 두고 이것을 조절

x,y 는 여전히 직접 예측